# BSGLMM Notebook

### Louis Soussand - Version 1.0 12/21/2017
Changelog:

1.0 Initial Release


### Steps to follow prior to the notebook:  
1) Go to [phenotypic datasets](http://nimlab-local-1:3838/apps/phenodat/), download your patients' data file   
2) Upload file to [dash_data](http://nimlab-local-1:3838/apps/dash_data/), create a control vs case, download file  
3) Upload lesions and filtered datasets to working directory from [xnat_gate](http://nimlab-local-1:3838/apps/xnat_gate/).  

### Steps within the notebook:  
1) Load lesion and phenotypic datasets  
2) Set up lesions folder for BSGLMM  
3) Create input file for BSGLMM  
4) Call BSGLMM using the call_bsglmm function from nimlab.software  
5) Look at the probability of type 1 and 2 given the data.  

In [1]:
#Packages and modules
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import subprocess
import shutil
import os
import sys
import pandas as pd
import numpy as np
from __future__ import unicode_literals
from __future__ import division
from nilearn import plotting
from time import time
import io
from time import time
import subprocess
import sys

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
import matplotlib.image as mpimg

from glob import glob
import nibabel as nib

sys.path.append('/data2/jupyter/notebooks/modules/')
from nimlab.functions import *
from nimlab.software import *

/data2/jupyter/notebooks/modules/envs/nimlab-python2-kernel/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# Input control:

#Load in dataframes
df_xnat=pd.read_csv("Corbetta_Lesions.csv")
df=pd.read_csv('Corbetta_Demographics_filtered_and_grouped.csv')



# Create target and control using output from dash_data
Control=df[df['trs_col_group()']=='Control']
Target=df[df['trs_col_group()']=='Case']

#Merge target and control with df_xnat using a left join.
Full_Target=Target.merge(df_xnat, on='subject', how='left')
Full_Control=Control.merge(df_xnat, on='subject', how='left')

#Dropping NAs patients:
TF=Full_Target.dropna(axis=0, how='any')
TC=Full_Control.dropna(axis=0, how='any')

# Combine target and control data frames:
dat=TF[['subject', 'sip_social', 'Lesion']].append(
TC[['subject', 'sip_social', 'Lesion']])
dat["type1"]=[1]*len(TF)+[0]*len(TC)
dat["type2"]=[0]*len(TF)+[1]*len(TC)
dat=dat.reset_index(drop=True)

In [ ]:
#Creating images folder for BSGLMM:

#Create folder and copy the BSGLMM mask to the local images folder
if not os.path.exists('images/'):
    os.makedirs('images/')
    shutil.copy('/data2/jupyter/notebooks/modules/BSGLMM_Support/Mask/mask.nii.gz', './images/')
#Copy all the lesions files for the analysis in the local images folder
for i in dat['Lesion']:
    shutil.copy(i, './images/')

#Extract the Lesion file names from the lesions paths:
outdf=[]
files = dat['Lesion']
for i in range(0,len(files)):
    chrstr=files[i]
    outstr=chrstr.split("/")[-1]
    outdf.append(outstr)

In [ ]:
# Create the BSGLMM input file as a dataframe:
datf=dat.rename(index=str, columns={"subject": "ID", "Lesion":"image_file"})

# Rename the columns with BSGLMM naming convention
datf=datf[['ID', 'type1', 'type2', 'sip_social', 'image_file']]

# Replace entire nimlab 
datf['image_file']=outdf

#Export the BSGLMM input file to the correct format
datf.to_csv("bsglmm_input.dat", index=None,sep=b"\t", encoding='utf-8')

In [ ]:
# Call BSGLMM using the call_bsglmm function from nimlab.software
call_bsglmm(2,3, MaxIter=1000000, BurnIn=500000)

In [ ]:
# Show BSGLMM outputs: 
from glob import glob
for i in glob("prb*"):
    cpgeom_cmd='fslcpgeom /usr/share/fsl/5.0/data/standard/MNI152_T1_2mm_brain.nii.gz '+i
    subprocess.call(cpgeom_cmd, shell=True)
    print "Printing "+i
    plotting.plot_glass_brain(i, display_mode='lyrz', colorbar=True)
    plotting.show()
    img = nib.load(i)
    plt.plot(np.ndarray.flatten(np.array(img.dataobj)))
    
for i in glob("standCoef*"):
    print "Printing "+i
    cpgeom_cmd='fslcpgeom /usr/share/fsl/5.0/data/standard/MNI152_T1_2mm_brain.nii.gz '+i
    subprocess.call(cpgeom_cmd, shell=True)
    plotting.plot_glass_brain(i, display_mode='lyrz', colorbar=True)
    plotting.show()
    img = nib.load(i)
    plt.plot(np.ndarray.flatten(np.array(img.dataobj)))